In [2]:
!pip -q install huggingface chromadb transformers langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 KB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.2/443.2 KB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 KB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [1]:
!pip install InstructorEmbedding

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from langchain.embeddings import HuggingFaceEmbeddings

In [4]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [5]:
hfEmbed = HuggingFaceEmbeddings()

In [6]:
hfInstructEmbed = HuggingFaceInstructEmbeddings(
 query_instruction="Represent the query for retrieval: "   
)

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
text = "This is a test document."

In [ ]:
query_result = hfEmbed.embed_query(text)

In [ ]:
instructor_result = hfInstructEmbed.embed_query(text)

In [7]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader

In [8]:
spaceLoad = TextLoader('/content/linux_play.txt')

In [9]:
from langchain.text_splitter import CharacterTextSplitter

In [10]:
documents = spaceLoad.load()

In [41]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/flan-ul2")

In [42]:
hfCharSplitter = CharacterTextSplitter.from_huggingface_tokenizer(tokenizer, 
                                                                  chunk_size=100,
                                                                  chunk_overlap=0)

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
recurSplitter = RecursiveCharacterTextSplitter(chunk_size=100,
                                               chunk_overlap=20,
                                               length_function=len)

In [43]:
char_documents = hfCharSplitter.split_documents(documents)

In [15]:
with open('/content/linux_play.txt') as lin:
  txt_lin = lin.read()

recurse_documents = recurSplitter.create_documents([txt_lin])

In [ ]:
recurse_documents[0]

Document(page_content='BEGIN;\nLinux Playbook', metadata={})

# Now work on embedding

In [44]:
from langchain.vectorstores import Chroma
# Supplying a persist_directory will store the embeddings on disk

charSplit_directory = 'charSplit_db'

plain_chroma = Chroma.from_documents(documents=char_documents,
                                     embeddings=hfEmbed,
                                     persist_directory=charSplit_directory)

In [36]:
# Supplying a persist_directory will store the embeddings on disk

recurse_directory = 'recurse_db'

recurse_chroma = Chroma.from_documents(documents=recurse_documents,
                                     embeddings=hfInstructEmbed,
                                     persist_directory=recurse_directory,
                                     collection_name='linux_playlist')

In [ ]:
query = "sed command"
plain_chroma.similarity_search(query)

In [ ]:
plain_chroma.similarity_search_with_score(query)

#Startin to use LLMs and Chains

In [47]:
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import AnalyzeDocumentChain
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

In [20]:
!pip -q install configparser

In [21]:
import os
import configparser

data = configparser.ConfigParser()
data.read_file(open('/content/apidata.config'))
os.environ['HUGGINGFACEHUB_API_TOKEN'] = data["OPENAI"]["HFREAD"]
os.environ['SERPAPI_API_KEY'] = data["OPENAI"]["SERPAPI_KEY"]
os.environ['OPENAI_API_KEY'] = data["OPENAI"]["KEY"]

In [48]:
from langchain.chains.summarize import load_summarize_chain

hugLLM = HuggingFaceHub(repo_id="google/flan-ul2",
                        model_kwargs={"temperature":0.1,
                                      "max_length":512})

summary_chain = load_summarize_chain(llm=hugLLM, 
                                     chain_type="map_reduce")

In [49]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=hugLLM, 
                                 chain_type="stuff", 
                                 retriever=plain_chroma.as_retriever())

In [51]:
qa.Config()

In [50]:
qa.run("How many Lines are there")

ValueError: ignored

In [52]:
from langchain.chains import LLMChain
prompt_template = """Use the context below to write a 400 word blog post about the topic below:
    Context: {context}
    Topic: {topic}
    Blog post:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "topic"]
)

chain = LLMChain(llm=hugLLM, 
                 prompt=PROMPT)

In [26]:
def generate_answer(topic):
    docs = recurse_chroma.similarity_search(topic, k=4)
    inputs = [{"context": doc.page_content, "topic": topic} for doc in docs]
    print(chain.apply(inputs))

In [27]:
generate_answer("Return all the Sed commands")

[{'text': 'sed is a stream editor that can be used to modify text files. It is a'}, {'text': "sed -e '/Cop/,$p' file sed -"}, {'text': "s/YEAR>/2022/' MIT.LICENSE # sed"}, {'text': "- sed commands: # sed -e 's/COPYRIGHT"}]


In [28]:
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo,
)

In [38]:
vectorstore_info = VectorStoreInfo(
    name="linux_playlist",
    description="the most recent linux playbook",
    vectorstore=recurse_chroma
)

In [34]:
from langchain.chains import RetrievalQAWithSourcesChain

In [37]:
toolkit = VectorStoreToolkit(llm=hugLLM,
                             vectorstore_info=vectorstore_info)

agent_executor = create_vectorstore_agent(
    llm=hugLLM,
    toolkit=toolkit,
    verbose=True
)

In [ ]:
agent_executor.run("How many lines are there in linux playbook")